<a href="https://colab.research.google.com/github/jdan98/bib/blob/master/IBM_final_task_2_data_analyze.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Write some markdown to explain that this notebook will be mainly used for the capstone project.



In [ ]:
import pandas as pd
import numpy as np
print ("hello capstone project")

hello capstone project
